In [ ]:
%matplotlib inline  
import numpy as np 
import seaborn as sns
import matplotlib.pylab as plt
import matplotlib.pyplot
#import seaborn as sn

In [ ]:
#attention_score = np.loadtxt('./shr17/s_v1/')
#attention_score = np.loadtxt('./shr17/s_v1/attention_eval.score.40273.0.txt')
#class_score = np.loadtxt('./shr17/s_v1/evaluate_testset_33696.0.score.txt')

#full train-val setup
attention_score = np.loadtxt('./shr17/s_v1/attention_eval.score.40273.0.txt')
class_score = np.loadtxt('./shr17/s_v1_fullt_train_val/model_best_testset_118420.0.score.txt')


#attention_score = np.loadtxt('./shr17/s_v1/attention_eval.score.51603.0.txt')
#class_score = np.loadtxt('./shr17/s_v1/evaluate_testset_33696.0.score.txt')

#attention_score = np.loadtxt('./shr17/v2/attention_eval.score.21690.0.txt')
#class_score = np.loadtxt('./shr17/v2/evaluate_testset_55510.0.score.txt')

#normal ring v1
#attention_score = np.loadtxt('./shr17/v2/attention_eval.score.21690.0.txt')
#class_score = np.loadtxt('./shr17/v1/evaluate_testset_27118.0.score.txt')

#normal ring v2
#full-train
#class_score = np.loadtxt('./shr17/v2_full_train/model_best_testset_165920.0.score.txt')
#partial train
#class_score = np.loadtxt('./shr17/v2/model_best_testset_42406.0.score.txt')
#attention_score = np.loadtxt('./shr17/v2/attention_score_valset/attention_eval.score.18090_thesis_by_valset.0.txt')

#attention_score = np.loadtxt('./shr17/v2/attention_score_trainset/eval.score.22022.0.txt')
#lass_score = np.loadtxt('./shr17/v2/model_best_testset_42406.0.score.txt')

#lstm
#class_score = np.loadtxt('./shr17/lstm_v2/evaluate_testset_32942.0.score.txt')

#attention_score = np.loadtxt('./shr17/s_v1/attention_eval.score.40273.0.txt')
#class_score = np.loadtxt('./shr17/v2_noshareweight/evaluate_testset_21112.0.score.txt')


#class_score = np.loadtxt('./shr17/s_v1_no_color/evaluate_testset_33696.0.score.txt')
#attention_score = np.loadtxt('./shr17/s_v1_no_color/attention_eval.score.40273.0.txt')

ground_truth_label = np.loadtxt('./test_real_label.txt')

#SHR18
#class_score = np.loadtxt('./shr18/square/evaluate_testset_103467.0.score.txt')
#attention_score = np.loadtxt('./shr18/square/attention/attention_eval.score.30121.0.txt')
#ground_truth_label = np.loadtxt('./shr18/test_real_label.txt')

#full train-val setup
#class_score = np.loadtxt('./shr18/square_train_val_full_fc/model_best_testset_110352.0.score.txt')
#attention_score = np.loadtxt('./shr18/square/attention/attention_eval.score.30121.0.txt')
#ground_truth_label = np.loadtxt('./shr18/test_real_label.txt')



N_VIEW_PER_RING =9
N_RING = 8

#N_VIEW_PER_RING = 8
#N_RING = 7

N_SAMPLE = ground_truth_label.shape[0]*1.

print N_SAMPLE

In [ ]:
attention_score = attention_score[:,1].reshape(-1,N_RING)
class_score = class_score.reshape(-1,N_RING,20)
print attention_score.shape
print class_score.shape

In [ ]:
# average all ring
preds = np.argmax(np.mean(class_score,1),1)
print sum(preds==ground_truth_label)/N_SAMPLE

avg_preds = preds


In [ ]:
print preds

In [ ]:
# choose best score ring
preds = []
for sample_index in range(attention_score.shape[0]):
    object_seven_ring_class_scores = class_score[sample_index] # (7x20)
    object_attention_score = attention_score[sample_index] # (7)
    
    #print object_attention_score
    k = np.argsort(object_attention_score)[-1]
    object_label = np.argmax(object_seven_ring_class_scores[k])
    preds.append(object_label)
print sum(preds==ground_truth_label)/N_SAMPLE
preds = []


In [ ]:
# sort from worst ring to best ring
for ring_attention_score_id in range(N_RING):
    preds = []
    for sample_index in range(attention_score.shape[0]):
        object_seven_ring_class_scores = class_score[sample_index] # (7x20)
        object_attention_score = attention_score[sample_index] # (7)

        #print object_attention_score
        k = np.argsort(object_attention_score)[ring_attention_score_id]
        object_label = np.argmax(object_seven_ring_class_scores[k])
        preds.append(object_label)
    print sum(preds==ground_truth_label)/N_SAMPLE
preds=[]

In [ ]:
# result by ring number k
for k in range(N_RING):
    preds = []
    for sample_index in range(attention_score.shape[0]):
        object_seven_ring_class_scores = class_score[sample_index] # (7x20)
        object_attention_score = attention_score[sample_index] # (7)

        #print object_attention_score4
        object_label = np.argmax(object_seven_ring_class_scores[k])
        preds.append(object_label)
    print sum(preds==ground_truth_label)/N_SAMPLE
preds=[]


In [ ]:
# choose top k best ring to predict
best_preds = []
best_acc = 0
for k in range(1,N_RING+1):
    preds = []
    for sample_index in range(attention_score.shape[0]):
        object_seven_ring_class_scores = class_score[sample_index] # (7x20)
        object_attention_score = attention_score[sample_index] # (7)

        #print object_attention_score
        q = np.argsort(object_attention_score)[::-1][:k]
        object_k_ring_class_scores = object_seven_ring_class_scores[q]
        pred = np.argmax(np.mean(object_k_ring_class_scores,0))
        preds.append(pred)
    acc = sum(preds==ground_truth_label)/N_SAMPLE
    if acc > best_acc:
        best_acc = acc
        best_preds = preds
    print 'top',k,':',acc


In [ ]:
print best_preds

In [ ]:
dif_predict = np.where( best_preds != avg_preds )
print dif_predict

In [ ]:
# weighted majority vote
preds=[]
for sample_index in range(int(N_SAMPLE)):
    object_seven_ring_class_scores = class_score[sample_index].copy() # (7x20)
    object_attention_score = attention_score[sample_index].copy() # (7)
    #print object_seven_ring_class_scores.shape
    #print object_attention_score.shape
    for i in range(N_RING):
        object_seven_ring_class_scores[i,:] *= object_attention_score[i]

    p = np.argmax(np.mean(object_seven_ring_class_scores,0))

    preds.append(p)
    
print sum(preds==ground_truth_label)/N_SAMPLE

weighted_preds = preds

In [ ]:
class_score[0][0].shape

In [ ]:
for i in range(8):
    fig=plt.figure()
    sn.barplot(range(20),class_score[19][i])

In [ ]:
class_names_shrec17[13]

In [ ]:
print weighted_preds

In [ ]:
np.where(avg_preds != weighted_preds)

In [ ]:
test_filename[11]

In [ ]:
attention_score[11]

In [ ]:
q = np.zeros((0,N_RING))
print q.shape
for i in range(427):
    grd_idx = ground_truth_label[i]
    p = class_score[i,:,int(grd_idx)]
    p = p.reshape(1,-1)
    q = np.vstack((q,p))
print q.shape


In [ ]:
for i in range(428//8+1):
    fig, ax =plt.subplots(1,2)
    sns.heatmap(q[i*8:i*8+8,:], linewidth=0.5, ax=ax[0], yticklabels=test_filename[i*8:i*8+8])
    sns.heatmap(attention_score[i*8:(i+1)*8,:], linewidth=0.5, ax=ax[1], yticklabels=[''])
    plt.show()

In [ ]:
q.shape

In [ ]:
# vote by rings that pass the threshold of k attention score
best_preds = 0
for k in np.arange(0,1,0.1):
    preds = []
    for sample_index in range(attention_score.shape[0]):
        #print sample_index
        
        
        object_seven_ring_class_scores = class_score[sample_index] # (7x20)
        object_attention_score = attention_score[sample_index] # (7)

        t = object_seven_ring_class_scores[object_attention_score>=k]
        #print t.shape
        if t.shape[0]==0:
            preds.append(0)
            continue

        p = np.argmax(np.mean(t,0))
        preds.append(p)
        
    #print 'pred length:',len(preds)
    print 'k=',k,'acc=', sum(preds==ground_truth_label)/427.0


In [ ]:
a = np.array([1,2,3,4,5,6])
b = np.array([0,0,0,3,3,3])
a[b>2]

In [ ]:
attention_score[0]

In [ ]:
a = np.array([1,2])
b = a
b[0] = 5

In [ ]:
a

In [ ]:
object_attention_score[[object_attention_score<0.5]]

# Visualize

In [ ]:
import seaborn as sns
import matplotlib.pylab as plt
import os
from matplotlib import figure

In [ ]:
test_filename = [u.split()[0] for u in open('./shr17/test2id.txt')]

In [ ]:
list_csv = [u.split() for u in open('./list.csv').readlines()[1:]]


In [ ]:
class_names_shrec17 = 'Bin, Box, Printer, Cup ,Light, Sofa ,Keyboard, Storage, Bed, Pillow, Machine, Bag, Book, Table, Oven, Desk ,Bookshelf,PCcase,Chair,Display'.split(',')
class_names_shrec18 ='chair	light	pc	table	cup	storage	desk	bag	display	bookshelf	bin	book	oven	bed	box	pillow	machine	printer	sofa	keyboard'.split('\t')
class_names_shrec17 = [x.strip() for x in class_names_shrec17]
class_names_shrec18 = [x.strip() for x in class_names_shrec18]

In [ ]:
import os

CMAP = plt.get_cmap('Reds')
def plot_square_ring(ring_number, obj_img_path, att_score, class_scores, real_label, class_names):
    LIST_RING = [
        [0, 1, 2, 8, 9, 10, 16, 17, 18],
        [1, 2, 3, 9, 10, 11, 17, 18, 19],
        [2, 3, 4, 10, 11, 12, 18, 19, 20],
        [3, 4, 5, 11, 12, 13, 19, 20, 21],
        [4, 5, 6, 12, 13, 14, 20, 21, 22],
        [5, 6, 7, 13, 14, 15, 21, 22, 23],
        [6, 7, 0, 14, 15, 8,  22, 23, 16],
        [7, 0, 1, 15, 8,  9,  23, 16, 17 ]
    ]

    
    r = LIST_RING[ring_number]
    img_paths =[os.path.join(obj_img_path, '{}.png'.format(rn)) for rn in r  ]
    #print obj_img_path
    w=8
    h=8
    fig=plt.figure(figsize=(w, h), facecolor=CMAP(att_score))
    columns = 3
    rows = 3
    for i in range(1, columns*rows +1):
        img = plt.imread(img_paths[i-1])
        fig.add_subplot(rows, columns, i)
        plt.axis('off')
        plt.imshow(img)
    
    
    save_img_paths =  obj_img_path.replace('shrec17_render','shrec17_render_square_att')
    if not os.path.exists(save_img_paths):
        os.makedirs(save_img_paths)
    
    
    #print class_names
    true_label = class_names[int(real_label)]
    
    #print class_scores.shape
    predict_label = class_names[np.argmax(class_scores)]
    
    title = 'a:{},p:{}'.format(true_label,predict_label)
    fig.suptitle(title, fontsize=20)

    #print save_img_paths
    plt.savefig(os.path.join(save_img_paths, '{}.png'.format(ring_number)), facecolor=fig.get_facecolor())

    #plt.show()
    plt.clf()
    plt.cla()
    plt.close('all')
    
    
    return img_paths

In [ ]:
import gc      

gc.collect()

In [ ]:
for obj_idx in range(401,ground_truth_label.shape[0]):
    print 'processing number ', obj_idx
    for k in range(8):
        t = plot_square_ring(k, '/home/minhb/Desktop/data/shrec17_render/{}.ply/'
                             .format(test_filename[obj_idx]), attention_score[obj_idx][k], class_score[obj_idx][k]
                            , ground_truth_label[obj_idx],
                             class_names_shrec17)
        


In [ ]:
attention_score[202]

In [ ]:
# sort from worst ring to best ring
for ring_attention_score_id in range(N_RING):
    preds = []
    for sample_index in range(attention_score.shape[0]):
        object_seven_ring_class_scores = class_score[sample_index] # (7x20)
        object_attention_score = attention_score[sample_index] # (7)

        #print object_attention_score
        k = np.argsort(object_attention_score)[ring_attention_score_id]
        object_label = np.argmax(object_seven_ring_class_scores[k])
        preds.append(object_label)
    print sum(preds==ground_truth_label)/427.0

In [ ]:
import seaborn as sn

In [ ]:
shrec17_class_labels = []

In [ ]:
Winter = plt.get_cmap('winter')

In [ ]:
Winter(0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cfs_matrix = confusion_matrix(y_pred=weighted_preds, y_true=ground_truth_label)
#cfs_matrix = confusion_matrix(y_pred=best_preds, y_true=ground_truth_label)

In [ ]:
class_names_shrec17

In [ ]:
import pandas as pd

In [ ]:
cm = cfs_matrix
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cfs_pd = pd.DataFrame(data=cm,columns=class_names_shrec18, index=class_names_shrec18)

In [ ]:
cfs_pd

In [ ]:
np.diag(cfs_pd)

In [ ]:
plt.figure(figsize=(10,8))
sn.heatmap(cfs_pd,cmap="YlGnBu")